# Processing scATAC Data

## Load Data

### Load packages

In [ ]:
import os
import numpy as np
import scanpy as sc
import pandas as pd
import anndata as ad
import episcanpy.api as epi
from matplotlib import pyplot as plt
# modules from gff-analyser
from gff_analyser import gffClasses, gffBuilder
# modules from sctoolbox
from sctoolbox import annotation
import sctoolbox.atac as atac
import sctoolbox.calc_overlap_pct as overlap
from sctoolbox import celltype_annotation
import hdf5plugin

In [ ]:
adata.write_h5ad(
    'stomach_feature.h5ad',
    
)

### Specify required paths

In [ ]:
GTF_PATH = '/home/stud3/notebooks/homo_sapiens.104.mainChr.gtf'  # gtf file to use for peaks annotation
GTF_ANALYSE = '/home/stud3/notebooks/out/homo_sapiens.104.mainChr_CDS_.gtf'
GTF_UROPA = '/home/stud3/notebooks/homo_sapiens.104.mainChr.gtf.gz'
INPUT_PATH = '/mnt/workspace_stud/catlas_ref/cellXcCRE/'  # path where the h5ad object is saved
OUTPUT_PATH = '/home/stud3/'  # path where processed adata file can be save
FRAG_FILE = '/home/stud3/notebooks/stomach_SM-JF1O3_rep1_fragments.bed.gz'  # path to fragments file
FRAG_FILE_sorted = '/home/stud3/notebooks/stomach_SM-JF1O3_rep1_fragments_sorted.bed'
BAM_FILE = ''  # path to bam file
test = '/home/stud3/notebooks/stomach_feature.h5ad'
test_ca = '/home/stud3/notebooks/stomach_test_ca.h5ad'
test_ca_uropa = '/home/stud3/notebooks/stomach_test_ca_uropa.h5ad'
H5AD = 'stomach_SM-JF1O3.h5ad'  # name of the h5ad file

In [ ]:
object_list = gffBuilder.build_gff3_class(file=GTF_PATH)

for element in object_list:
    features = element.count_features()

    element.generate_feature_gtf(Gffdata_list=object_list, feature_keys=features)

## Read and customize data

### Read Anndata object

In [ ]:
adata = epi.read_h5ad(INPUT_PATH + H5AD)
adata

### inspect adata.var

In [ ]:
adata = epi.read_h5ad(test)

#### Adjust peak names
* if the features names are not in the format chr_start_end, reformat them to avoid issues in downstream analysis

In [ ]:
names = []
for name in adata.var_names:
    names.append(name)

for i, n in enumerate(names):
    tmp = names[i].replace(':', '_')
    tmp = tmp.replace('-', '_')
    tmp = tmp.split('_')
    names[i] = '_'.join([tmp[0], tmp[-2], tmp[-1]])

adata.var_names = pd.Index(names)

### inspect adata.obs

In [ ]:
adata.obs

### Calculate mean insertsize

In [ ]:
adata.obs

In [ ]:
names = []
for name in adata.obs_names:
    names.append(name)

for i, n in enumerate(names): # colon_sigmoid_SM-JF1O8_1+
    tmp = names[i].split('+')
    names[i] = tmp[-1]

adata.obs_names = pd.Index(names, name='index')

In [ ]:
atac.add_insertsize(adata, fragments=FRAG_FILE_sorted)

In [ ]:
# plot insert size
atac.plot_insertsize(adata)

In [ ]:
adata

###  Promotor enrichment 
* To speed up calculation, use fragments file instead of a bam file and set bam_file=None
* If cell barcodes are not in index, specify column name using parameter cb_col
* Specify species: [homo_sapiens, mus_musculus, danio_rerio,...]

In [ ]:
species = 'homo_sapiens'
#frag files müssen unzipped werden und danach sortiert 
overlap.pct_fragments_in_features(adata, input_dir='out/', fragments_file=FRAG_FILE_sorted, bam_file=None, 
                                   cb_col=None, species=None)

In [ ]:
# remove cells with empty features
epi.pp.filter_cells(adata, min_features=1)
# remove features with no cells
epi.pp.filter_features(adata, min_cells=1)

## QC

In [ ]:
adata.obs['nb_features']

### Remove chrM

In [ ]:
non_m = [name for name in adata.var_names if not name.startswith('chrM')]
adata = adata[:, non_m]
display(adata)

### Optional: Remove chrX and chrY

In [ ]:
non_xy = [name for name in adata.var_names if not name.startswith('chrY') | name.startswith('chrX')]
adata = adata[:, non_xy]
display(adata)

### Remove cells without features or empty features

In [ ]:
# remove cells with empty features
epi.pp.filter_cells(adata, min_features=1)
# remove features with no cells
epi.pp.filter_features(adata, min_cells=1)

# calculate the log of the number of features in each cell
adata.obs['log_nb_features'] = [np.log10(x) for x in adata.obs['nb_features']]
adata

### Binarize matrix and save different layers

In [ ]:
adata.raw = adata

In [ ]:
epi.pp.binarize(adata)
adata.layers['binary'] = adata.X.copy()

## Filter Data

In [ ]:
adata.obs['nb_features']

### Promoter enrichment

In [ ]:
# plot promoter enrichment
sc.pl.violin(adata, keys = ['nb_features'], groupby = None, rotation=90)

In [ ]:
# filter cells based on percentage of fragments in promoters
adata = adata[adata.obs['log_nb_features'] > 0.2]

### Visualize feature distribution (Histogram)

In [ ]:
# show open features per cell
min_features = 100

epi.pp.coverage_cells(adata, binary=True, log=False, bins=50,
               threshold=min_features)
epi.pp.coverage_cells(adata, binary=True, log=10, bins=50,
               threshold=min_features)

### Visualize feature distribution (Violin)

In [ ]:
epi.pl.violin(adata, ['nb_features'])
epi.pl.violin(adata, ['log_nb_features'])

### Filter cells

In [ ]:
# filter cells which have at least min_features and at most max_features
epi.pp.filter_cells(adata, min_features=150)
epi.pp.filter_cells(adata, max_features=10000)

In [ ]:
# filter features that appear in at least min_cells and at most max_cells
epi.pp.filter_features(adata, min_cells=10)
epi.pp.filter_features(adata, max_cells=200)

### Visualize distribution of cells sharing a feature

In [ ]:
# show numbers of cells sharing features
min_cells = 10

epi.pp.coverage_features(adata, binary=True, log=False, bins=50,
               threshold=min_cells)
epi.pp.coverage_features(adata, binary=True, log=10, bins=50,
               threshold=min_cells)

### Filter features

### Visualize feature distribution after filtering

In [ ]:
# visualize
epi.pl.violin(adata, ['nb_features'])
epi.pl.violin(adata, ['log_nb_features'])

In [ ]:
# show open features per cell
min_features = 100

epi.pp.coverage_cells(adata, binary=True, log=False, bins=50,
               threshold=min_features)
epi.pp.coverage_cells(adata, binary=True, log=10, bins=50,
               threshold=min_features)

# show numbers of cells sharing features
min_cells = 10

epi.pp.coverage_features(adata, binary=True, log=False, bins=50,
               threshold=min_cells)
epi.pp.coverage_features(adata, binary=True, log=10, bins=50,
               threshold=min_cells)

# calculate varibaility score
epi.pp.cal_var(adata)

In [ ]:
adata

### Normalize remaining data

In [ ]:
sc.pp.normalize_total(adata)
adata.layers['normalised'] = adata.X.copy()

# log-normalize
epi.pp.log1p(adata)

## Dimension reduction and clustering

### Calculate PCA and neighbors

In [ ]:
# calculate pca
sc.pp.pca(adata, n_comps=50, svd_solver='arpack', use_highly_variable=False)
# calculate neighbors
sc.pp.neighbors(adata, n_neighbors=15, n_pcs=50, method='umap', metric='euclidean')

In [ ]:
# Plot PCA variance ratio for selection of PCs
sc.pl.pca_variance_ratio(adata, n_pcs = 30)

In [ ]:
adata.obs

### Calculate UMAP

In [ ]:
sc.tl.umap(adata, min_dist=0.5, spread=2.5)

In [ ]:
# Visualize cells in UMAP
sc.pl.umap(adata, color = 'nb_features', legend_loc = 'right margin')

### Cluster with leiden algorithm and show UMAP

In [ ]:
sc.tl.leiden(adata, resolution=2.5, use_weights=False)

sc.pl.umap(adata, color=['leiden'])

## Peaks Annotation

### UROPA

In [ ]:
custom_config = {"queries": [{"feature": 'gene', "distance": [5000, 5000], "feature_anchor": "start"}],
                 "priority": True, 
                 "show_attributes": "all"}

In [ ]:
annotation.annotate_adata(adata, gtf=GTF_UROPA, config=custom_config, best=True, threads=3, coordinate_cols=None, temp_dir="", remove_temp=True, verbose=True, inplace=True)

In [ ]:
adata.var

#### filter unassigned peaks in uropa

In [ ]:
assigned_peaks = adata.var[adata.var['gene_name'].notnull()]
uropa_adata = adata[:,assigned_peaks.index]
uropa_adata

### Replace peaks with gene names
Make new feature names unique and write them into raw

In [ ]:
uropa_adata.var.reset_index(inplace=True)
uropa_adata.var.set_index('gene_id', inplace=True)

In [ ]:
uropa_adata.var.index = uropa_adata.var.index.astype('object')

In [ ]:
uropa_adata.var_names_make_unique(join="_")
uropa_adata.raw = uropa_adata
uropa_adata.var

### Rank genes

In [ ]:
sc.tl.rank_genes_groups(uropa_adata, groupby='leiden', use_raw=True)

sc.pl.rank_genes_groups(uropa_adata)

In [ ]:
sc.pl.rank_genes_groups_matrixplot(uropa_adata, standard_scale='var', n_genes=10)

In [ ]:
uropa_adata.var.index

## Celltype annotation

In [ ]:
celltype_annotation.run_scsa(uropa_adata, species='human')

### Visulaize with UMAP

In [ ]:
# Visualize cells in UMAP
sc.pl.umap(uropa_adata, color = 'SCSA_pred_celltype', title = 'Predicted Celltypes', legend_loc = 'right margin')

In [ ]:
uropa_adata